# Grocery List Generation
The Recepticon application helps users find new recipes for items they already buy at the grocery store with no or minimal substitutions. One component of the app is allowing users the ability to populate a list of groceries that the system will then take into account into the recipe optimizer. 

We are going to write a class that will work with a `.csv` file containing a list of ingredients and do the following:

* Return results from a query
* Add item to list
* Delete item from list
* Create a new list
* Delete a list
* Load a list

This should all work similarly to an online grocery cart, although for the sake of keeping things simple it will ignore quantity and dollar amount.

In [59]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import _pickle as pickle
import uuid
import random
%matplotlib inline

In [2]:
ingredients = pd.read_csv('../git_main/python/files/ingr_list.csv')
ingredients.head()

,Unnamed: 0,ingredient,red_meat,poultry,fish,seafood,egg,vegetables,fruits,dairy
0,0,maitake mushrooms,0,0,0,0,0,1,0,0
1,1,america,0,0,0,0,0,0,0,0
2,2,shiro miso,0,0,0,0,0,0,0,0
3,3,hash brown,0,0,0,0,0,0,0,0
4,4,jicama,0,0,0,0,0,0,0,0


## Search
First piece we'll tackle is running a search. The class will be running on the back of a flask app. The front-end of the app will be passing a 'query' to the python code so we will assume that the query input as already been generated.

In [13]:
query = 'broccoli'
#ingredients[ingredients['ingredient'].str.contains(query)==True]
ingredients['ingredient'][ingredients['ingredient'].str.contains(query)==True]

454           broccolini
823        broccoli slaw
1352    broccoli florets
1803    chopped broccoli
1869            broccoli
2368       broccoli rabe
Name: ingredient, dtype: object

#### Display in HTML
We want to output this list in HTML including an icon for each item allowing the user to add to list. We'll test some HTML functionality here to make sure the output looks like how we want.

In [4]:
## import HTML display modeul for jupyter notebooks - awesome !!
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))

In [24]:
def grocery_row_gen(ingr):
    entry = '''<tr>
    <td>%s</td>
    <td><button type="button" class="btn btn-default btn-lg">
  <span class="glyphicon glyphicon-plus-sign" aria-hidden="true"></span> Add to Current List
</button></td>
  </tr>''' %ingr
    #display(HTML(entry))
    return (entry)
grocery_row_gen(query)

'<tr>\n    <td>broccoli</td>\n    <td><button type="button" class="btn btn-default btn-lg">\n  <span class="glyphicon glyphicon-plus-sign" aria-hidden="true"></span> Add to Current List\n</button></td>\n  </tr>'

### Adding HTML to Search Function
We've figured out how to search the ingredients list and how to generate an HTML table. Now we can generate a table from the function.

In [25]:
def ingr_search(query, series):
    query_results = series[series.str.contains(query)==True]
    html_string = '''<table>
    <tr>
    <th>Item</th>
    <th>Click to add</th>
    </tr>'''
    for row in query_results:
        #print(row)
        html_string += grocery_row_gen(row)
    html_string += """</table>"""
    return (html_string)

In [26]:
display(HTML(ingr_search(query, ingredients['ingredient'])))

Item,Click to add
broccolini,Add to Current List
broccoli slaw,Add to Current List
broccoli florets,Add to Current List
chopped broccoli,Add to Current List
broccoli,Add to Current List
broccoli rabe,Add to Current List


## Grocery List Class
Each user in our system will have grocery lists associated with their accounts. They will be able to create new lists and load saved lists from their account as desired. We'll leverage object-oriented programming in Python to do this. First we'll establish a new class and then build functionality into it.

In [53]:
class GroceryList:
    def __init__(self, name, user_id):
        self.name = name
        self.user_id = user_id
        self.groc_list = [] # initialize with empty list of items
        self.filename = uuid.uuid4().hex + '.pkl'
    
    def add_item(self, item):
        self.groc_list.append(item)
        print(item + " has been added to " + self.name + ".")
    
    def delete_item(self, item):
        self.groc_list.remove(item)
        print(item + " has been removed from " + self.name + ".")
    
    def get_items(self):
        print("Current items in " + self.name + ":")
        for item in self.groc_list:
            print(item)
            
    def save_list(self):
        with open(self.filename, 'wb') as output:  # Overwrites any existing file.
            pickle.dump(self, output, -1)
            
    def clear_list(self):
        self.groc_list = []

In [54]:
mylist = GroceryList("mylist", "chet")
mylist.add_item('broccoli')
mylist.add_item('orange')
mylist.add_item('chicken')
mylist.get_items()
mylist.delete_item('orange')
mylist.get_items()

broccoli has been added to mylist.
orange has been added to mylist.
chicken has been added to mylist.
Current items in mylist:
broccoli
orange
chicken
orange has been removed from mylist.
Current items in mylist:
broccoli
chicken


## Storing Object Data
For this to be useful to us, we'll need to store object data so that it can be opened again. This is a very simple, stripped down shopping cart, so there's nothing too complicated about the data that's being passed around. Per the following post from Stack Overflow, we'll `pickle` the data and save to files that can be opened and closed using the pickle utility in Python:

https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence

To make the best use of this, we'll need to add a filename associated with each list. We can leverage the *tempfile* utility built into Python to generate a random file name when a new class object is created.

*Note: We will also need to store the filename somewhere outside of the object so that it can be called and loaded from file*

In [55]:
mylist.save_list()

Our class object *mylist* should now be saved as `mylist.pkl` in the local folder. To test that this worked properly, we're going to delete the list object *mylist* and then load using the *load_list* method that we just created.

In [56]:
del mylist
print(mylist.getitems())

NameError: name 'mylist' is not defined

In [47]:
def load_list(filename):
    with open(filename, 'rb') as input:
        current_list = pickle.load(input)
        return current_list

In [57]:
mylist = load_list('048d0da4881a4d12ae3f212c231b97a5.pkl')
mylist.get_items()

Current items in mylist:
broccoli
chicken


Let's make sure that this works as expected when we modify a list, save it again, and then bring it back.

In [58]:
mylist.delete_item('chicken')
mylist.add_item('smoothie')
mylist.add_item('Cheetos')
mylist.get_items()
mylist.save_list()
del mylist
mylist = load_list('048d0da4881a4d12ae3f212c231b97a5.pkl')
mylist.get_items()

chicken has been removed from mylist.
smoothie has been added to mylist.
Cheetos has been added to mylist.
Current items in mylist:
broccoli
smoothie
Cheetos
Current items in mylist:
broccoli
smoothie
Cheetos


### Having Some Fun
Now that the grocery list has all the functions we want, it's time to have a little fun. We're going to write a function that will generate a new shopping list with 20 items. We'll do this a couple times and then check that we have different files for each list and they can be re-loaded into Python.

In [67]:
def randomGroceryList(num, list_name, user_id, ingredients):
    ingr = set(ingredients)
    tmp_obj = GroceryList(list_name, user_id)
    print(tmp_obj.filename)
    tmp_items = random.sample(ingr, num)
    for itm in tmp_items:
        tmp_obj.add_item(itm)
    tmp_obj.save_list()

In [68]:
randomGroceryList(20, 'mylist1', 'chet', ingredients['ingredient'])

096d1bddcbf449ad878fcb247f7ae989.pkl
egg roll wrappers has been added to mylist1.
root has been added to mylist1.
calamata olives has been added to mylist1.
dough has been added to mylist1.
cactus has been added to mylist1.
tapioca has been added to mylist1.
smithfield ham has been added to mylist1.
thin pizza crust has been added to mylist1.
asian rice noodles has been added to mylist1.
ginger root has been added to mylist1.
glutinous rice flour has been added to mylist1.
cherry pie filling has been added to mylist1.
sweet sherry has been added to mylist1.
sangiovese has been added to mylist1.
sour orange juice has been added to mylist1.
caramel sauce has been added to mylist1.
yellow onion has been added to mylist1.
chinese celery cabbage has been added to mylist1.
epazote has been added to mylist1.
seltzer water has been added to mylist1.
